# ITSLIVE Screener: UTM zone 45N

##as of 12/1 night this is working for all rgi regions!

Want to find the glaciers in HMA that are flowing fast enough that we could use itslive to examine them...

In [1]:
!pwd

/uufs/chpc.utah.edu/common/home/u1269862/2022/agu_22_processing


In [2]:
import xarray as xr 
import numpy as np
import pandas as pd
import geopandas as gpd 
import rioxarray as rio
import rioxarray.merge as merge
import urllib.request
import json
import os
from osgeo import gdal
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
# skimage.morphology import skeletonize
#import itslivetools
#from itslivetools import itslive_setup

In [3]:
rio.__version__

'0.13.1'

In [4]:
#set epsg code
epsg_code = '32645'
epsg_code_full = f'EPSG:{epsg_code}'
zone_num = int(epsg_code[-2:])

print(epsg_code)
print(epsg_code_full)
print(zone_num)

32645
EPSG:32645
45


In [5]:
os.chdir('/uufs/chpc.utah.edu/common/home/u1269862/2022')
dir_path = '/uufs/chpc.utah.edu/common/home/u1269862/2022/'

In [6]:
#read in setup data
os.chdir('/uufs/chpc.utah.edu/common/home/u1269862/2022')
utm_grid = gpd.read_file('World_UTM_Grid(1).geojson')
rgi13 = gpd.read_file(dir_path + 'rgi/13_rgi60_CentralAsia.shp')
rgi14 = gpd.read_file(dir_path + 'rgi/14_rgi60_SouthAsiaWest.shp')
rgi15 = gpd.read_file(dir_path + 'rgi/15_rgi60_SouthAsiaEast.shp')
sa_13_ablationlines = gpd.read_file(dir_path + '/zhang_lower_centerlines/13_rgi60_CentralAsia_Ablationlines.shp').to_crs('EPSG:4326')
sa_14_ablationlines = gpd.read_file(dir_path + '/zhang_lower_centerlines/14_rgi60_SouthAsiaWest_Ablationlines.shp').to_crs('EPSG:4326')
sa_15_ablationlines = gpd.read_file(dir_path + '/zhang_lower_centerlines/15_rgi60_SouthAsiaEast_Ablationlines.shp').to_crs('EPSG:4326')

In [7]:
rgi_combine = pd.concat([rgi13, rgi14, rgi15])
rgi_combine = rgi_combine.loc[rgi_combine['Area'] >= 5.]

In [8]:
#read in itslive catalog
catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

In [9]:
#find all granules in a given utm zone
def find_urls_by_utm(url_ls, epsg_code):
    
    code = 'EPSG' + epsg_code
    url_idx = [url for url in range(len(url_ls)) if code in url_ls[url]]
    url_subset = [url_ls[i] for i in url_idx]
        
    return url_subset

## Data prep



In [10]:
#subset utm grid to single zone (full latitude span)
utm_zone= utm_grid.loc[utm_grid['ZONE'] == zone_num]

### Lower glacier flowlines

1. each (region 13, 14, 15) are in different projections converted all 3 to latlon on read in 
2. for each of them, need to select only the glaciers that lie in the single utm zone
3. need to separately reproject the subsetted centerlines dataframes to local utm

In [11]:
sa_13_lines_zone = gpd.sjoin(sa_13_ablationlines, utm_zone, how='inner')
sa_14_lines_zone = gpd.sjoin(sa_14_ablationlines, utm_zone, how='inner')
sa_15_lines_zone = gpd.sjoin(sa_15_ablationlines, utm_zone, how='inner')

In [11]:
#fig, ax = plt.subplots()
#utm_zone.plot(ax=ax, facecolor='None')
#sa_13_ablationlines.plot(ax=ax, facecolor='None', alpha = 0.1, color='gray')
#sa_14_ablationlines.plot(ax=ax, facecolor='None', edgecolor='gray', alpha = 0.1)
#sa_15_ablationlines.plot(ax=ax, facecolor='None', edgecolor='gray', alpha = 0.1)

#rgi_zone.plot(ax=ax, facecolor='None', edgecolor='purple')

#sa_13_lines_zone.plot(ax=ax, facecolor='None')
#sa_14_lines_zone.plot(ax=ax, facecolor='None', edgecolor='green')
#sa_15_lines_zone.plot(ax=ax, facecolor='None', edgecolor='red')

#ax.set_ylim(25,50);

In [12]:
sa_13_lines_zone = sa_13_lines_zone.to_crs(epsg_code_full)
sa_14_lines_zone = sa_14_lines_zone.to_crs(epsg_code_full)
sa_15_lines_zone = sa_15_lines_zone.to_crs(epsg_code_full)

In [13]:
flowlines_combined = pd.concat([sa_13_lines_zone, sa_15_lines_zone])

In [14]:
flowlines_combined.head()

,GLIMS_ID,Type,MaxL,MeanL,ELA,AccL,AblationL,geometry,index_right,FID,ZONE,ROW_,WEST_VALUE,CM_VALUE,EAST_VALUE
4723,G089819E28234N,1,512.0,512.0,5538,284.0,227.0,"LINESTRING (776644.839 3126314.919, 776644.849...",386,387,45,R,84E,87E,90E
4724,G084355E28854N,1,737.0,737.0,5925,350.0,387.0,"LINESTRING (241923.724 3194719.006, 241919.067...",386,387,45,R,84E,87E,90E
4725,G084361E28854N,1,849.0,815.0,5862,365.0,483.0,"LINESTRING (242470.787 3194602.465, 242470.646...",386,387,45,R,84E,87E,90E
4726,G084331E28856N,2,6300.0,4515.0,5851,4370.0,1930.0,"LINESTRING (238467.889 3194156.998, 238465.097...",386,387,45,R,84E,87E,90E
25974,G084522E28832N,1,1305.0,1305.0,6086,844.0,461.0,"LINESTRING (258083.448 3191933.035, 258073.780...",386,387,45,R,84E,87E,90E


### RGI outlines
Need to
1. subset RGI to footprint of UTM zone, then 
2. project from latlon to local UTM, 
3. make list of the glims ids of the rgi glaciers
4. subset RGI dataframe to only the glaciers for which there is also a centerline


In [15]:
#fig, ax = plt.subplots(figsize=(12,10))
#rgi_combine.plot(ax=ax, color= 'red', alpha = 0.2)
#rgi_zone.plot(ax=ax)


In [15]:
#subset rgi to just the utm zone
rgi_zone = gpd.sjoin(rgi_combine, utm_zone, how='inner')

In [16]:
#project to local utm
rgi_zone_prj = rgi_zone.to_crs(epsg_code_full)

In [17]:
#make a list of the glims ids of the centerlines dataframe
centerlines_glims_ids = flowlines_combined['GLIMS_ID'].to_list()

In [18]:
#subset rgi df to only include glaciers for which there is also centerline data
rgi_zone_w_centerlines = rgi_zone.loc[rgi_zone['GLIMSId'].isin(centerlines_glims_ids)]

In [19]:
rgi_zone_w_centerlines = rgi_zone_w_centerlines.drop('index_right', axis=1)

In [20]:
len(rgi_zone_w_centerlines)

275

In [21]:
#now subset itslive granules to only those with target glaciers in it
itslive_granules_zone = gpd.sjoin(catalog, rgi_zone_w_centerlines, how='inner')

In [21]:
#itslive_granules_zone.plot(facecolor='None')

In [22]:
url_strs = list(itslive_granules_zone['zarr_url'].astype(str))
urls_zone = find_urls_by_utm(url_strs, epsg_code)

In [23]:
itslive_granules_zone = itslive_granules_zone[['fill-opacity','fill','roi_percent_coverage','geometry_epsg','datacube_exist','zarr_url','epsg','geometry']]

In [24]:
url_strs = list(set(list(itslive_granules_zone['zarr_url'].astype(str))))

In [25]:
urls_zone = find_urls_by_utm(url_strs, epsg_code)

In [26]:
urls_zone

['http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E080/ITS_LIVE_vel_EPSG32645_G0120_X350000_Y3750000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E080/ITS_LIVE_vel_EPSG32645_G0120_X450000_Y4150000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E080/ITS_LIVE_vel_EPSG32645_G0120_X550000_Y3550000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E080/ITS_LIVE_vel_EPSG32645_G0120_X550000_Y4150000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N20E080/ITS_LIVE_vel_EPSG32645_G0120_X250000_Y3150000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N40E080/ITS_LIVE_vel_EPSG32645_G0120_X350000_Y4750000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E080/ITS_LIVE_vel_EPSG32645_G0120_X650000_Y3950000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N20E080/ITS_LIVE_vel_EPSG32645_G0120_X550000_Y3050000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E080/ITS_LIVE_vel_EPSG3

In [27]:
#now subset urls list to only the target granules, hopefuly this speeds things up 

In [28]:
rgi_zone_w_centerlines_prj = rgi_zone_w_centerlines.to_crs(epsg_code_full)
#rgi_zone_w_centerlines_prj = rgi_zone_w_centerlines_prj.drop('index_right', axis=1)

In [31]:
#fig, ax = plt.subplots(figsize=(8,6))

#utm_zone.plot(facecolor='None',ax=ax)
#itslive_granules_zone.plot(facecolor='None', ax=ax)
#rgi_zone_w_centerlines.plot(ax=ax, color='green')
#ax.set_ylim(20,32)
#ax.set_xlim(66,74)

### ITSLIVE data

In [29]:
def read_in_s3(http_url, chunks = 'auto'):
    s3_url = http_url.replace('http','s3')
    s3_url = s3_url.replace('.s3.amazonaws.com','')

    datacube = xr.open_dataset(s3_url, engine = 'zarr',
                                storage_options={'anon':True},
                                chunks = chunks)

    return datacube

In [30]:
def get_bbox(input_xr, epsg=None):
    
    '''Takes input xr object (from itslive data cube), plots a quick map of the footprint. 
    currently only working for granules in crs epsg 32645'''

    xmin = input_xr.coords['x'].data.min()
    xmax = input_xr.coords['x'].data.max()

    ymin = input_xr.coords['y'].data.min()
    ymax = input_xr.coords['y'].data.max()

    pts_ls = [(xmin, ymin), (xmax, ymin),(xmax, ymax), (xmin, ymax), (xmin, ymin)]

   
    crs = f"epsg:{epsg}"
    #print(crs)
    
    polygon_geom = Polygon(pts_ls)
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 
    #polygon = polygon.to_crs('epsg:4326')

    return polygon


In [31]:
#dc_test = read_in_s3(urls_zone[0])
#dc_test = dc_test.rio.write_crs(epsg_code_full)

In [32]:
#bbox_test = get_bbox(dc_test, epsg=epsg_code)
#bbox_test

In [31]:
len(urls_zone)

36

In [32]:
class IndGlacier:
    
    def __init__(self, rgi_id, rgi_outline, itslive, ablationLine = None): #removed dem, centerline
        
        self.rgi_id = rgi_id
        self.rgi_outline = rgi_outline
        #self.dem = dem
        self.itslive = itslive
        #self.centerLine = centerLine
        self.ablationLine = ablationLine

    def __getitem__(self, key):
        return getattr(self, key)
    
    def __setitem__(self,key,value):
        return getattr(self,key, value)


In [35]:
def ind_glacier_data_prep(rgi_id, rgi_full, itslive_dc, lower_centerlines, utm_code): #dem_obj,dem_obj,
    '''function to prepare data to create an object of the `IndGlacier` class for a single glacier.
    Pass in RGI ID of glacier of interest as well as full data objects of RGI gpdf, itslive datacube, 
    nasadem and the u tm code fo the glacier.I feel like at scale this is probably a really inefficient way to build objects'''
    
    #clip rgi to glacier
    single_rgi = rgi_full.loc[rgi_full['RGIId'] == rgi_id]
    #print(single_rgi['RGIId'])
    #extract glims id - will use to extract centerlines
    glims_id = single_rgi['GLIMSId'].values
    #print(glims_id)
    
    
    #clip dem and itslive
    #dem_clip = dem_obj.rio.clip(single_rgi.geometry, dem_obj.rio.crs).squeeze().transpose()
    itslive_clip = itslive_dc.rio.clip(single_rgi.geometry, itslive_dc.rio.crs)
    #dem_clip_downsamp = dem_clip.interp_like(itslive_clip, method = 'nearest')
    #itslive_clip['z'] = dem_clip_downsamp.Band1
    #for now, this data will be subsetted to only obs in the same season! 
    itslive_clip = itslive_clip.where(itslive_clip.acquisition_date_img1.dt.season == itslive_clip.acquisition_date_img2.dt.season, drop=True)
    
    #centerline objects
    #centerline = centerlines.loc[centerlines['GLIMS_ID'] == glims_id[0]].to_crs(utm_code)
    lower_centerline = lower_centerlines.loc[lower_centerlines['GLIMS_ID'] == glims_id[0]].to_crs(utm_code)
    
    #flowline object
    
    #centerline = centerline.to_crs(utm_code)
    #ablationline = ablationline.to_crs(utm_code)
    #lowestpoint = lowestpoints.to_crs(
    
    #creat object of IndGlacier class
    rgi_outline = single_rgi
    glacier = IndGlacier(rgi_id, rgi_outline, itslive_clip, lower_centerline) #dem_clip_downsamp, centerline
    
    return glacier

The function below *should* make df of median speed for each season for each glacier in each element of the list of itslive granules

In [36]:
def single_granule_prep(single_granule_url, rgi_zone_prj, centerlines, epsg_code_full):
        
    if epsg_code_full[5:] in single_granule_url:

        dc = read_in_s3(single_granule_url)
        dc = dc.rio.write_crs(epsg_code_full)
        dc_bbox=get_bbox(dc, epsg = epsg_code)

        if str(dc_bbox.crs)[5:] == str(rgi_zone_prj.crs)[5:]:
            #print('granule and rgi crs match')
            pass
        else:
            print('crs issue')

        rgi_sub_zone = gpd.sjoin(rgi_zone_prj, dc_bbox, how='inner')
            #make lists of glacier ids (RGI + GLIMS) for all glaciers in the set of interest
        rgi_zone_ids = rgi_sub_zone['RGIId'].to_list()
        glims_zone_ids = rgi_sub_zone['GLIMSId'].to_list()

            #Subset the centerlines dataset to only the glaciers in the target list:
        centerlines_zone = flowlines_combined.loc[flowlines_combined['GLIMS_ID'].isin(glims_zone_ids)]
            #project centerlines to local utm
        centerlines_zone_prj = centerlines_zone.to_crs(epsg_code_full)

            #The centerline dataset might not always have all of the glaciers either, need to update the other lists: <br>
        rgi_sub_zone = rgi_sub_zone.loc[rgi_sub_zone['GLIMSId'].isin(centerlines_zone['GLIMS_ID'])]
            #add area threshold
        rgi_sub_zone = rgi_sub_zone.loc[rgi_sub_zone['Area'] >= 5.]

        rgi_zone_ids = rgi_sub_zone['RGIId'].to_list()
        glims_zone_ids = rgi_sub_zone['GLIMSId'].to_list()
        print('# of glaciers in granule: ', len(rgi_zone_ids))

            #now building full glacier objects for every glacier
            #ind_granule_df = make_glacier_objects(rgi_zone_ids, rgi_sub_zone, dc, sa_central_ablationlines, epsg_code_full)
            #ind_granule_df.to_csv(f'/home/emmamarshall/Desktop/phd_work/itslive/median_speeds_dfs/{granule}_df.csv')
            #df_ls.append(ind_granule_df)
        #print('data is prepped')

        return dc, rgi_sub_zone, rgi_zone_ids, glims_zone_ids
    else:
        print('this glacier is probably on a granule or utm zone border and the url returned isnt in the utm zone you specified')
    

In [45]:
already_made_dfs = os.listdir(f'/uufs/chpc.utah.edu/common/home/u1269862/2022/screener_dfs_{epsg_code}')


In [41]:
#rgi_zone_w_centerlines_prj = rgi_zone_w_centerlines_prj.drop('index_right', axis=1)

In [37]:
gran_ls_32645 = [single_granule_prep(urls_zone[gran], rgi_zone_w_centerlines_prj, flowlines_combined, epsg_code_full) for gran in range(len(urls_zone))]

# of glaciers in granule:  6
# of glaciers in granule:  4
# of glaciers in granule:  2
# of glaciers in granule:  24
# of glaciers in granule:  6
# of glaciers in granule:  23
# of glaciers in granule:  1
# of glaciers in granule:  5
# of glaciers in granule:  1
# of glaciers in granule:  1
# of glaciers in granule:  6
# of glaciers in granule:  4
# of glaciers in granule:  4
# of glaciers in granule:  29
# of glaciers in granule:  1
# of glaciers in granule:  1
# of glaciers in granule:  1
# of glaciers in granule:  1
# of glaciers in granule:  1
# of glaciers in granule:  3
# of glaciers in granule:  4
# of glaciers in granule:  11
# of glaciers in granule:  1
# of glaciers in granule:  2
# of glaciers in granule:  4
# of glaciers in granule:  1
# of glaciers in granule:  36
# of glaciers in granule:  2
# of glaciers in granule:  5
# of glaciers in granule:  35
# of glaciers in granule:  30
# of glaciers in granule:  1
# of glaciers in granule:  35
# of glaciers in granule:  11
# of 

In [41]:
type(gran_ls_32645[0])
gran_ls_32645[0][0]

<xarray.Dataset>
Dimensions:                    (mid_date: 11474, y: 833, x: 834)
Coordinates:
    mapping                    int64 0
  * mid_date                   (mid_date) datetime64[ns] 2021-04-23T05:18:41....
  * x                          (x) float64 3.001e+05 3.002e+05 ... 4e+05
  * y                          (y) float64 4.1e+06 4.1e+06 ... 4e+06 4e+06
Data variables: (12/53)
    acquisition_date_img1      (mid_date) datetime64[ns] dask.array<chunksize=(11474,), meta=np.ndarray>
    acquisition_date_img2      (mid_date) datetime64[ns] dask.array<chunksize=(11474,), meta=np.ndarray>
    autoRIFT_software_version  (mid_date) <U5 dask.array<chunksize=(11474,), meta=np.ndarray>
    chip_size_height           (mid_date, y, x) float32 dask.array<chunksize=(11474, 50, 50), meta=np.ndarray>
    chip_size_width            (mid_date, y, x) float32 dask.array<chunksize=(11474, 50, 50), meta=np.ndarray>
    date_center                (mid_date) datetime64[ns] dask.array<chunksize=(11474,), meta=np.ndarray>
    ...                         ...
    vy_error_mask              (mid_date) float64 dask.array<chunksize=(11474,), meta=np.ndarray>
    vy_error_modeled           (mid_date) float64 dask.array<chunksize=(11474,), meta=np.ndarray>
    vy_error_slow              (mid_date) float64 dask.array<chunksize=(11474,), meta=np.ndarray>
    vy_stable_shift            (mid_date) float64 dask.array<chunksize=(11474,), meta=np.ndarray>
    vy_stable_shift_mask       (mid_date) float64 dask.array<chunksize=(11474,), meta=np.ndarray>
    vy_stable_shift_slow       (mid_date) float64 dask.array<chunksize=(11474,), meta=np.ndarray>
Attributes: (12/18)
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               09-Jun-2022 05:17:59
    date_updated:               09-Jun-2022 05:17:59
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v02/N30E080/ITS_...
    skipped_granules:           s3://its-live-data/datacubes/v02/N30E080/ITS_...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image_pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

In [46]:
def calc_glacier_medians_1granule(granule_obj, centerlines, epsg_code_full, already_made_ls):
#format of granule obj is [0] - dc, [1] - rgi gpdf, [2] - list of rgi ids, [3] - list of glims ids
    
    dc = granule_obj[0]
    rgi_sub_zone = granule_obj[1]
    rgi_zone_ids = granule_obj[2]
    
    rgi_ids, med_speeds_spring, med_speeds_summer, med_speeds_fall, med_speeds_winter = [],[], [], [],[]

    for element in range(len(rgi_zone_ids)):
        rgiid = rgi_zone_ids[element][6:].replace('.','_')
        dfname = f'{rgiid}_medv_df.csv'
        #print(rgiid)

        
        if dfname in already_made_ls:
            
            print('no need')
        
        else:
                
            try:   
                glacier_obj = ind_glacier_data_prep(rgi_zone_ids[element], rgi_sub_zone, dc, centerlines, epsg_code_full)
                #print(glacier_obj.ablationLine)
                #print('check')
                glacier_its = glacier_obj.itslive.compute()
                #print('check 2')
                glacier_gb = glacier_its.groupby(glacier_its.mid_date.dt.season).median(skipna=True)
                #glaciers_gb.append(glacier_gb)
                # glaciers_full.append(glacier_obj)
                # ids.append(glacier_obj.rgi_id)
                #print(glacier_obj.rgi_id , ' object constructed')
                #try: 
                flowline_obj = glacier_gb.rio.clip(glacier_obj.ablationLine.geometry, glacier_gb.v.rio.crs)
                #print('flowline made')
                del glacier_its

                fl = flowline_obj.v.compute()
                #print('flowline computed')
                fl_med_spring = fl.sel(season='MAM').median(skipna=True).data
                fl_med_summer = fl.sel(season='JJA').median(skipna=True).data
                fl_med_fall = fl.sel(season='SON').median(skipna=True).data
                fl_med_winter = fl.sel(season='DJF').median(skipna=True).data

                del fl

                rgi_ids.append(rgi_zone_ids[element])

                #if fl_med_spring >= 50.:

                med_speeds_spring.append(fl_med_spring.data)
                #elif fl_med_summer >= 50.:
                med_speeds_summer.append(fl_med_summer.data)
                #elif fl_med_fall >= 50.:
                med_speeds_fall.append(fl_med_fall.data)
                #elif fl_med_winter >= 50.:
                med_speeds_winter.append(fl_med_winter.data)
                med_speeds_dict = {'RGI_ID':rgi_zone_ids[element], 'Med_v_spring': fl_med_spring, 'med_v_summer':fl_med_summer,
                                                      'med_v_fall':fl_med_fall, 'med_v_winter':fl_med_winter}
                med_speeds_df = pd.DataFrame(med_speeds_dict, index=['RGI_ID'])
                rgiid = rgi_zone_ids[element][-8:].replace('.','_')
                med_speeds_df.to_csv(f'/uufs/chpc.utah.edu/common/home/u1269862/2022/screener_dfs_{epsg_code}/{rgiid}_medv_df.csv')
                print(rgi_zone_ids[element][-8:].replace('.','_'))
            except:
                print('issue w this: ', rgi_zone_ids[element])
            

In [ ]:


for element in range(len(gran_ls_32645)):
    
    print(element)
    
    if len(gran_ls_32645[element][1]['RGIId'].to_list()) > 0:
        
        calc_glacier_medians_1granule(gran_ls_32645[element], flowlines_combined, epsg_code_full,already_made_dfs)
    else:
        pass

0
13_36475
13_36513
issue w this:  RGI60-13.36536
issue w this:  RGI60-13.36613
issue w this:  RGI60-13.36620
issue w this:  RGI60-13.54026
1
no need
no need
no need
no need
2
no need
no need
3
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
4
no need
no need
no need
no need
no need
no need
5
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
no need
issue w this:  RGI60-15.09991
no need
6
no need
7
no need
no need
